In [37]:
import os
import sys
from os.path import join
from tqdm import tqdm

import pandas as pd
import numpy as np
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [38]:
ICLOUD_PATH = r'/Users/lukeneuendorf/Library/Mobile Documents/com~apple~CloudDocs/bdb25'
df_tracking = pd.read_pickle(join(ICLOUD_PATH, 'data', 'tracking_w1.pickle'))
df_play = pd.read_pickle(join(ICLOUD_PATH, 'data', 'play_w1.pickle'))

In [39]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def plot_play(df_tracking, game_play_id, every_other_frame=True, event_col='event'):
    qry = 'game_play_id==@game_play_id'
    tracking_play = df_tracking.query(qry).copy().reset_index(drop=True)

    # Kepe every other frame, the first and last frames, and frames with events
    first_frame = tracking_play['frame_id'].min()
    last_frame = tracking_play['frame_id'].max()
    frames_with_events = tracking_play.groupby('frame_id')[event_col].transform('any')

    if every_other_frame:
        tracking_play = tracking_play[
            (tracking_play['frame_id'] == first_frame) | 
            (tracking_play['frame_id'] == last_frame) | 
            (frames_with_events) |
            (tracking_play['frame_id'] % 2 == 0)  # Keep even frames only
        ].copy().reset_index(drop=True)
    else:
        tracking_play = tracking_play[
            (tracking_play['frame_id'] == first_frame) | 
            (tracking_play['frame_id'] == last_frame) | 
            (frames_with_events)
        ].copy().reset_index(drop=True)

    frames = tracking_play['frame_id'].unique()
    current_event = [None]  

    field_width = 53.3

    fig, ax = plt.subplots(figsize=(10, 5))

    padding = 2
    min_y = tracking_play.y.min() - padding
    max_y = tracking_play.y.max() + padding

    los = tracking_play['absolute_yardline_number'].iloc[0]
    to_go_line = los + tracking_play['yards_to_go'].iloc[0]

    def update(frame_id):
        """Update function for each animation frame."""
        ax.clear()

        # can you make the field a light grey color?
        ax.set_facecolor('lightgrey')
        
        ax.set_yticks(np.arange(10, 110+1, 5))
        ax.grid(which='major', axis='y', linestyle='-', linewidth='0.5', color='black', zorder=1)

        for spine in ax.spines.values():
            spine.set_visible(False)

        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

        ax.set_xlim(0, field_width)
        ax.set_ylim(min_y, max_y)

        current_frame = tracking_play.query('frame_id == @frame_id')
        offense = current_frame.query('offense')
        defense = current_frame.query('~offense and club != "football"')
        football = current_frame.query('club == "football"')

        # plot the los in blue
        ax.axhline(los, color='blue', linewidth=1.2, linestyle='-', zorder=1)

        # plot the line to go
        ax.axhline(to_go_line, color='yellow', linewidth=1.2, linestyle='-', zorder=1)

        # event = current_frame[event_col].iloc[0] if not current_frame[event_col].isna().all() else None
        # if event:
        #     current_event[0] = event

        ax.scatter(offense.x, offense.y, c='red', edgecolor='black', label='Offense', zorder=2)
        ax.scatter(defense.x, defense.y, c='blue', edgecolor='black', label='Defense', zorder=2)
        ax.scatter(football.x, football.y, c='brown', edgecolor='black', label='Football', s=20, zorder=3)

        event = current_frame[event_col].iloc[0]
        box_color = 'white'  # Default color
        alpha_value = 0.8    # Transparency value
        
        # Set box color based on event type
        if event == 'line_set':
            box_color = 'green'
            alpha_value = 0.5  # More transparency for the green box
        elif event == 'ball_snap':
            box_color = 'red'
            alpha_value = 0.5  # More transparency for the red box
        
        ax.text(
            1, max_y + 1.5, f"{event}",
            fontsize=12, ha='left', color='black',
            bbox=dict(facecolor=box_color, alpha=alpha_value),
            zorder=4
        )

        ax.text(52.3, max_y + 1.5, f"{frame_id / 10:.01f} s", fontsize=12, ha='right', color='black', 
                bbox=dict(facecolor='white', alpha=0.8), zorder=4)

    ani = FuncAnimation(fig, update, frames=frames, interval=100, repeat=False)

    plt.subplots_adjust(left=0, right=1, bottom=0, top=.9)

    plt.close(fig)

    return HTML(ani.to_jshtml(fps=5))

In [13]:
plot_play(df_tracking, '2022091200_64', 'event_new')

In [14]:
df_tracking.head()

,game_id,play_id,game_play_id,nfl_id,week,display_name,frame_id,frame_type,time,jersey_number,club,play_direction,x,y,s,a,dis,o,dir,event,position,absolute_yardline_number,yards_to_go,offense,defense,ball_x,ball_y,euclidean_dist_to_ball,lateral_dist_to_ball,vertical_dist_to_ball
0,2022091200,64,2022091200_64,35459.0,1,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,22.0,DEN,right,24.75,51.06,0.72,0.37,0.07,293.83,111.66,huddle_break_offense,SS,40,10,False,True,29.429999,39.470001,12.499219,-4.679999,11.589999
1,2022091200,64,2022091200_64,35459.0,1,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,22.0,DEN,right,24.73,51.13,0.71,0.36,0.07,294.59,108.79,NaN,SS,40,10,False,True,29.429999,39.480000,12.562345,-4.699999,11.650000
2,2022091200,64,2022091200_64,35459.0,1,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,22.0,DEN,right,24.71,51.20,0.69,0.23,0.07,295.55,110.10,NaN,SS,40,10,False,True,29.429999,39.480000,12.634746,-4.719999,11.720000
3,2022091200,64,2022091200_64,35459.0,1,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,22.0,DEN,right,24.68,51.26,0.67,0.22,0.07,295.55,112.02,NaN,SS,40,10,False,True,29.429999,39.480000,12.701610,-4.749999,11.780000
4,2022091200,64,2022091200_64,35459.0,1,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,22.0,DEN,right,24.65,51.32,0.65,0.34,0.07,294.26,117.17,NaN,SS,40,10,False,True,29.429999,39.480000,12.768477,-4.779999,11.840000


In [17]:
df_play.head()

,game_id,play_id,game_play_id,play_description,quarter,down,yards_to_go,possession_team,defensive_team,yardline_side,yardline_number,game_clock,pre_snap_home_score,pre_snap_visitor_score,play_nullified_by_penalty,absolute_yardline_number,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,expected_points,offense_formation,receiver_alignment,play_clock_at_snap,pass_result,pass_length,target_x,target_y,play_action,dropback_type,dropback_distance,pass_location_type,time_to_throw,time_in_tackle_box,time_to_sack,pass_tipped_at_line,unblocked_pressure,qb_spike,qb_kneel,qb_sneak,rush_location_type,penalty_yards,pre_penalty_yards_gained,yards_gained,home_team_win_probability_added,visitor_team_win_probility_added,expected_points_added,is_dropback,pff_run_concept_primary,pff_run_concept_secondary,pff_run_pass_option,pff_pass_coverage,pff_man_zone,play_direction
0,2022091104,3662,2022091104_3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,12:51,28,38,N,45,0.078611,0.921389,-0.141130,SHOTGUN,3x1,3.0,I,-6.0,44.69,10.53,False,TRADITIONAL,1.78,INSIDE_BOX,1.568,1.568,NaN,True,True,False,0,NaN,NaN,NaN,0,0,0.012361,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,right
1,2022091112,1674,2022091112_1674,(:35) (Shotgun) A.Rodgers pass deep left inten...,2,1,10,GB,MIN,GB,25,00:35,17,0,N,35,0.914143,0.085857,0.602453,EMPTY,4x1,3.0,IN,45.0,75.33,43.82,False,SCRAMBLE,2.63,OUTSIDE_LEFT,4.506,3.500,NaN,False,False,False,0,NaN,NaN,NaN,0,0,0.003262,-0.003262,-1.071627,True,NaN,NaN,0,Cover-2,Zone,right
2,2022091111,923,2022091111_923,(11:59) J.Herbert pass deep right to K.Allen t...,2,1,10,LAC,LV,LAC,47,11:59,3,3,N,57,0.629273,0.370727,2.447006,SINGLEBACK,2x2,11.0,C,28.0,36.73,38.69,True,TRADITIONAL,6.69,INSIDE_BOX,3.537,3.537,NaN,False,False,False,0,NaN,NaN,NaN,42,42,0.048007,-0.048007,2.483530,True,INSIDE ZONE,NaN,0,Cover-3,Zone,left
3,2022091109,3544,2022091109_3544,(11:04) (Shotgun) C.Wentz pass short right to ...,4,2,7,WAS,JAX,WAS,25,11:04,14,22,N,35,0.186782,0.813218,0.617924,SHOTGUN,3x1,16.0,C,-4.0,88.92,36.31,True,TRADITIONAL,0.87,INSIDE_BOX,1.468,1.468,NaN,False,False,False,0,NaN,NaN,NaN,-1,-1,-0.028717,0.028717,-1.009146,True,OUTSIDE ZONE,NaN,1,Cover-6 Right,Zone,left
4,2022091109,2502,2022091109_2502,(8:54) C.Van Lanen reported in as eligible. T...,3,2,3,JAX,WAS,WAS,3,08:54,14,3,N,107,0.751983,0.248017,5.818538,SINGLEBACK,2x1,4.0,C,3.0,10.10,44.41,True,DESIGNED_ROLLOUT_RIGHT,5.31,OUTSIDE_RIGHT,2.970,1.600,NaN,False,False,False,0,NaN,NaN,NaN,3,3,-0.027986,0.027986,1.181462,True,MAN,NaN,0,Red Zone,Other,left


In [26]:
df_pbp = nfl.import_pbp_data([2022])
df_pbp['old_game_id_x'] = df_pbp.old_game_id_x.astype(int)
df_pbp['play_id'] = df_pbp.play_id.astype(int)

2022 done.
Downcasting floats.


In [40]:
df_pbp = nfl.import_pbp_data([2022])
df_pbp['old_game_id_x'] = df_pbp.old_game_id_x.astype(int)
df_pbp['play_id'] = df_pbp.play_id.astype(int)

pbp_cols = ['old_game_id_x','play_id','play_type','qb_scramble','run_location','run_gap']
rename_dict = {
    'old_game_id_x': 'game_id'
}
if 'play_type' not in df_play.columns:
    df_play = df_play.merge(
        df_pbp[pbp_cols].rename(rename_dict, axis=1),
        on=['game_id','play_id'],
        how='left'
    )

In [41]:
df_play.head()

,game_id,play_id,game_play_id,play_description,quarter,down,yards_to_go,possession_team,defensive_team,yardline_side,yardline_number,game_clock,pre_snap_home_score,pre_snap_visitor_score,play_nullified_by_penalty,absolute_yardline_number,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,expected_points,offense_formation,receiver_alignment,play_clock_at_snap,pass_result,pass_length,target_x,target_y,play_action,dropback_type,dropback_distance,pass_location_type,time_to_throw,time_in_tackle_box,time_to_sack,pass_tipped_at_line,unblocked_pressure,qb_spike,qb_kneel,qb_sneak,rush_location_type,penalty_yards,pre_penalty_yards_gained,yards_gained,home_team_win_probability_added,visitor_team_win_probility_added,expected_points_added,is_dropback,pff_run_concept_primary,pff_run_concept_secondary,pff_run_pass_option,pff_pass_coverage,pff_man_zone,play_direction,play_type,qb_scramble,run_location,run_gap
0,2022091104,3662,2022091104_3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,12:51,28,38,N,45,0.078611,0.921389,-0.141130,SHOTGUN,3x1,3.0,I,-6.0,44.69,10.53,False,TRADITIONAL,1.78,INSIDE_BOX,1.568,1.568,NaN,True,True,False,0,NaN,NaN,NaN,0,0,0.012361,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,right,pass,0.0,None,None
1,2022091112,1674,2022091112_1674,(:35) (Shotgun) A.Rodgers pass deep left inten...,2,1,10,GB,MIN,GB,25,00:35,17,0,N,35,0.914143,0.085857,0.602453,EMPTY,4x1,3.0,IN,45.0,75.33,43.82,False,SCRAMBLE,2.63,OUTSIDE_LEFT,4.506,3.500,NaN,False,False,False,0,NaN,NaN,NaN,0,0,0.003262,-0.003262,-1.071627,True,NaN,NaN,0,Cover-2,Zone,right,pass,0.0,None,None
2,2022091111,923,2022091111_923,(11:59) J.Herbert pass deep right to K.Allen t...,2,1,10,LAC,LV,LAC,47,11:59,3,3,N,57,0.629273,0.370727,2.447006,SINGLEBACK,2x2,11.0,C,28.0,36.73,38.69,True,TRADITIONAL,6.69,INSIDE_BOX,3.537,3.537,NaN,False,False,False,0,NaN,NaN,NaN,42,42,0.048007,-0.048007,2.483530,True,INSIDE ZONE,NaN,0,Cover-3,Zone,left,pass,0.0,None,None
3,2022091109,3544,2022091109_3544,(11:04) (Shotgun) C.Wentz pass short right to ...,4,2,7,WAS,JAX,WAS,25,11:04,14,22,N,35,0.186782,0.813218,0.617924,SHOTGUN,3x1,16.0,C,-4.0,88.92,36.31,True,TRADITIONAL,0.87,INSIDE_BOX,1.468,1.468,NaN,False,False,False,0,NaN,NaN,NaN,-1,-1,-0.028717,0.028717,-1.009146,True,OUTSIDE ZONE,NaN,1,Cover-6 Right,Zone,left,pass,0.0,None,None
4,2022091109,2502,2022091109_2502,(8:54) C.Van Lanen reported in as eligible. T...,3,2,3,JAX,WAS,WAS,3,08:54,14,3,N,107,0.751983,0.248017,5.818538,SINGLEBACK,2x1,4.0,C,3.0,10.10,44.41,True,DESIGNED_ROLLOUT_RIGHT,5.31,OUTSIDE_RIGHT,2.970,1.600,NaN,False,False,False,0,NaN,NaN,NaN,3,3,-0.027986,0.027986,1.181462,True,MAN,NaN,0,Red Zone,Other,left,pass,0.0,None,None


In [48]:
df_pbp.qb_kneel.value_counts()

0.0    48991
1.0      443
Name: qb_kneel, dtype: int64